<a href="#" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Subset Selection using GradMatch

In this tutorial, we will cover the basics of cords using GradMatch strategy by training on [MNIST dataset](http://yann.lecun.com/exdb/mnist/)

---
  - Give us a ⭐ [on Github](https://github.com/decile-team/cords)
  - Check out [the documentation](https://cords.readthedocs.io/en/latest/index.html)

In [ ]:
!pip install -i https://test.pypi.org/simple/ cords

In [ ]:
import time
import datetime
import copy
import numpy as np
import os
import subprocess
import sys
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim as optim
from matplotlib import pyplot as plt
from torch.utils.data.sampler import SubsetRandomSampler
from cords.selectionstrategies.supervisedlearning import OMPGradMatchStrategy
from cords.utils.models import MnistNet
from cords.utils.custom_dataset import load_dataset_custom
from torch.utils.data import random_split, SequentialSampler, BatchSampler, RandomSampler, Subset
from torch.autograd import Variable
import math
import tqdm

In [ ]:
def model_eval_loss(data_loader, model, criterion):
    total_loss = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(data_loader):
            inputs, targets = inputs.to(device), targets.to(device, non_blocking=True)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item()
    return total_loss

### Model Creation

In [ ]:
def create_model(name, num_cls, device):
  elif name == 'MnistNet':
     model = MnistNet()
  model = model.to(device)
  return model

### Define Loss Type, Optimizer and Learning Rate Scheduler

In [ ]:
def loss_function():
  criterion = nn.CrossEntropyLoss()
  criterion_nored = nn.CrossEntropyLoss(reduction='none')
return criterion, criterion_nored

In [ ]:
def optimizer_with_scheduler(model, num_epochs, learning_rate, m=0.9, wd=5e-4):
    optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                          momentum=m, weight_decay=wd)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
    return optimizer, scheduler

In [ ]:
def generate_cumulative_timing(mod_timing):
  tmp = 0
  mod_cum_timing = np.zeros(len(mod_timing))
  for i in range(len(mod_timing)):
    tmp += mod_timing[i]
    mod_cum_timing[i] = tmp
  return mod_cum_timing/3600

In [ ]:
from scipy.signal import lfilter
def filter(y):
  n = 1 # the larger n is, the smoother curve will be
  b = [1.0 / n] * n
  a = 1
  yy = lfilter(b,a,y)
  return yy

Implementation of OMPGradMatch Strategy from the paper (under review) for supervised learning frameworks.

OMPGradMatch strategy tries to solve the optimization problem given below:
\begin{equation*}
        \min_{\mathbf{w}, S: |S| \leq k} \Vert \sum_{i \in S} w_i \nabla_{\theta}L_T^i(\theta) -  \nabla_{\theta}L(\theta)\Vert
\end{equation*}

In the above equation, ${\mathbf{w}}$ denotes the weight vector that contains the weights for each data instance, ${\mathcal{U}}$ training set where ${(x^i, y^i)}$ denotes the ${i^{th}}$ training data point and label respectively, ${L_T}$ denotes the training loss, ${L}$ denotes either training loss or validation loss depending on the parameter valid, ${S}$ denotes the data subset selected at each round, and :math:`k` is the budget for the subset.

The above optimization problem is solved using the Orthogonal Matching Pursuit(OMP) algorithm.


---------------------------------------
#### Parameters for GradMatch Strategy

    trainloader: class
        Loading the training data using pytorch DataLoader
    valloader: class
        Loading the validation data using pytorch DataLoader
    model: class
        Model architecture used for training
    loss_type: class
        The type of loss criterion
    eta: float
        Learning rate. Step size for the one step gradient update
    device: str
        The device being utilized - cpu | cuda
    num_classes: int
        The number of target classes in the dataset
    linear_layer: bool
        Apply linear transformation to the data
    selection_type: str
        Type of selection -
        - 'PerClass': PerClass method is where OMP algorithm is applied on each class data points seperately.
        - 'PerBatch': PerBatch method is where OMP algorithm is applied on each minibatch data points.
        - 'PerClassPerGradient': PerClassPerGradient method is same as PerClass but we use the gradient corresponding to classification layer of that class only.
    valid : bool, optional
        If valid==True we use validation dataset gradient sum in OMP otherwise we use training dataset (default: False)
    lam : float
        Regularization constant of OMP solver
    eps : float
        Epsilon parameter to which the above optimization problem is solved using OMP algorithm

### General Training Loop with GradMatch Strategy

In [ ]:
def train_model(num_epochs, dataset_name, datadir, feature, model_name, fraction, select_every, learning_rate, run, device, strategy):
  
    #Loading the Dataset
    trainset, validset, testset, num_cls = load_dataset_custom(datadir, dataset_name, feature)  
    N = len(trainset)
    trn_batch_size = 20
    val_batch_size = 1000
    tst_batch_size = 1000

    #Creating the Data Loaders
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=trn_batch_size,
                                            shuffle=False, pin_memory=True)

    valloader = torch.utils.data.DataLoader(validset, batch_size=val_batch_size, 
                                          shuffle=False, pin_memory=True)

    testloader = torch.utils.data.DataLoader(testset, batch_size=tst_batch_size,
                                            shuffle=False, pin_memory=True)


    #Budget for subset selection
    if strategy != "Full":
      bud = int(fraction * N)
      print("Budget, fraction and N:", bud, fraction, N)
      #Subset Selection and creating the subset data loader
      start_idxs = np.random.choice(N, size=bud, replace=False)
      idxs = start_idxs
      data_sub = Subset(trainset, idxs)
      subset_trnloader = torch.utils.data.DataLoader(data_sub, batch_size=trn_batch_size, 
                                                    shuffle=False, pin_memory=True)

      #Variables to store accuracies
      gammas = torch.ones(len(idxs)).to(device)    
      substrn_losses = np.zeros(num_epochs)
      subtrn_acc = np.zeros(num_epochs)

    val_losses = np.zeros(num_epochs)
    timing = np.zeros(num_epochs)
    val_acc = np.zeros(num_epochs)
    tst_acc = np.zeros(num_epochs)
    
    #Results logging file
    if strategy == 'Full':
      all_logs_dir = '/content/drive/MyDrive/Results/' + dataset_name + '/' + strategy + '/' + feature  
    else:
      all_logs_dir = '/content/drive/MyDrive/Results/' + dataset_name + '/' + strategy + '/' + feature +'/' + str(fraction) + '/' + str(select_every)
    
    print(all_logs_dir)
    subprocess.run(["mkdir", "-p", all_logs_dir])
    path_logfile = os.path.join(all_logs_dir, dataset_name + '_run_' + str(run + 1) + '.txt')
    logfile = open(path_logfile, 'w')
    exp_name = dataset_name + '_fraction:' + str(fraction) + '_epochs:' + str(num_epochs) + \
              '_selEvery:' + str(select_every) + '_variant' + '_runs' + str(run)
    print(exp_name)

    #Model Creation
    model = create_model(model_name, num_cls, device)
    model1 = create_model(model_name, num_cls, device)
    #Loss Functions
    criterion, criterion_nored = loss_function()

    #Getting the optimizer and scheduler
    optimizer, scheduler = optimizer_with_scheduler(model, num_epochs, learning_rate)

    if strategy == 'GradMatch':
        #OMPGradMatch Selection strategy
        setf_model = OMPGradMatchStrategy(trainloader, valloader, model1, criterion,
                                learning_rate, device, num_cls, True, 'PerClassPerGradient',
                                  False, lam=0.5, eps=1e-100)
    elif strategy == 'GradMatchPB':
        setf_model = OMPGradMatchStrategy(trainloader, valloader, model1, criterion,
                                learning_rate, device, num_cls, True, 'PerBatch',
                                  False, lam=0, eps=1e-100)
    elif strategy == 'GradMatch-Explore':
        #OMPGradMatch Selection strategy
        setf_model = OMPGradMatchStrategy(trainloader, valloader, model1, criterion,
                                learning_rate, device, num_cls, True, 'PerClassPerGradient',
                                  False, lam=0.5, eps=1e-100)
        #Random-Online Selection strategy
        rand_setf_model = RandomStrategy(trainloader, online=True)
    elif strategy == 'GradMatchPB-Explore':
        #OMPGradMatch Selection strategy
        setf_model = OMPGradMatchStrategy(trainloader, valloader, model1, criterion,
                                learning_rate, device, num_cls, True, 'PerBatch',
                                  False, lam=0, eps=1e-100)
        #Random-Online Selection strategy
        rand_setf_model = RandomStrategy(trainloader, online=True)

    print("=======================================", file=logfile) 
    
    for i in tqdm.trange(num_epochs):
        subtrn_loss = 0
        subtrn_correct = 0
        subtrn_total = 0
        subset_selection_time  = 0
        if (strategy in ['GradMatch', 'GradMatchPB']) and (((i+1) % select_every) == 0):
            start_time = time.time()
            cached_state_dict = copy.deepcopy(model.state_dict())
            clone_dict = copy.deepcopy(model.state_dict())
            subset_idxs, gammas = setf_model.select(int(bud), clone_dict)
            model.load_state_dict(cached_state_dict)
            idxs = subset_idxs
            if strategy in ['GradMatch', 'GradMatchPB']:
                gammas = torch.from_numpy(np.array(gammas)).to(device).to(torch.float32)
            subset_selection_time += (time.time() - start_time)
        elif (strategy in ['GradMatch-Explore', 'GradMatchPB-Explore']) :
            start_time = time.time()
            if i < 99:
                subset_idxs, gammas = rand_setf_model.select(int(bud))
                idxs = subset_idxs
                gammas = gammas.to(device)
            elif ((i+1)% select_every == 0):
                cached_state_dict = copy.deepcopy(model.state_dict())
                clone_dict = copy.deepcopy(model.state_dict())
                subset_idxs, gammas = setf_model.select(int(bud), clone_dict)
                model.load_state_dict(cached_state_dict)
                idxs = subset_idxs
                if strategy in ['GradMatch-Explore', 'GradMatchPB-Explore']:
                    gammas = torch.from_numpy(np.array(gammas)).to(device).to(torch.float32)
            subset_selection_time += (time.time() - start_time)

        if strategy != "Full":
          print("selEpoch: %d, Selection Ended at:" % (i), str(datetime.datetime.now()))
          data_sub = Subset(trainset, idxs)
          subset_trnloader = torch.utils.data.DataLoader(data_sub, batch_size=trn_batch_size, shuffle=False,
                                        pin_memory=True)
        
        model.train()

        if strategy in ['GradMatch', 'GradMatch-Explore', 'GradMatchPB', 'GradMatchPB-Explore']:
          batch_wise_indices = list(subset_trnloader.batch_sampler)
          start_time = time.time()
          for batch_idx, (inputs, targets) in enumerate(subset_trnloader):
              inputs, targets = inputs.to(device), targets.to(device, non_blocking=True) # targets can have non_blocking=True.
              optimizer.zero_grad()
              outputs = model(inputs)
              losses = criterion_nored(outputs, targets)
              loss = torch.dot(losses, gammas[batch_wise_indices[batch_idx]])/(gammas[batch_wise_indices[batch_idx]].sum())
              loss.backward()
              subtrn_loss += loss.item()
              optimizer.step()
              _, predicted = outputs.max(1)
              subtrn_total += targets.size(0)
              subtrn_correct += predicted.eq(targets).sum().item()
          train_time = time.time() - start_time
        elif strategy in ['Full']:
          start_time = time.time()
          for batch_idx, (inputs, targets) in enumerate(trainloader):
              inputs, targets = inputs.to(device), targets.to(device, non_blocking=True) # targets can have non_blocking=True.
              optimizer.zero_grad()
              outputs = model(inputs)
              loss = criterion(outputs, targets)
              loss.backward()
              subtrn_loss += loss.item()
              optimizer.step()
              _, predicted = outputs.max(1)
              subtrn_total += targets.size(0)
              subtrn_correct += predicted.eq(targets).sum().item()
          train_time = time.time() - start_time
        scheduler.step()
        timing[i] = train_time + subset_selection_time
        #print("Epoch timing is: " + str(timing[i]))
        val_loss = 0
        val_correct = 0
        val_total = 0
        tst_correct = 0
        tst_total = 0
        tst_loss = 0
        model.eval()
        
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(valloader):
                #print(batch_idx)
                inputs, targets = inputs.to(device), targets.to(device, non_blocking=True)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_total += targets.size(0)
                val_correct += predicted.eq(targets).sum().item()

            for batch_idx, (inputs, targets) in enumerate(testloader):
                #print(batch_idx)
                inputs, targets = inputs.to(device), targets.to(device, non_blocking=True)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                tst_loss += loss.item()
                _, predicted = outputs.max(1)
                tst_total += targets.size(0)
                tst_correct += predicted.eq(targets).sum().item()

        val_acc[i] = val_correct/val_total
        tst_acc[i] = tst_correct/tst_total
        val_losses[i] = val_loss

        if strategy != 'Full':
          subtrn_acc[i] = subtrn_correct/subtrn_total
          substrn_losses[i] = subtrn_loss
        
        print('Epoch:', i + 1, 'Validation Accuracy: ', val_acc[i], 'Test Accuracy: ', tst_acc[i], 'Time: ',  timing[i])
    print(strategy + " Selection Run---------------------------------")
    print("Final SubsetTrn:", subtrn_loss)
    print("Validation Loss and Accuracy:", val_loss, val_acc.max())
    print("Test Data Loss and Accuracy:", tst_loss, tst_acc.max())
    print('-----------------------------------')
    
    #Results logging into the file
    print(strategy, file=logfile)
    print('---------------------------------------------------------------------', file=logfile)
    val = "Validation Accuracy, "
    tst = "Test Accuracy, "
    time_str = "Time, "
    
    for i in range(num_epochs):
        time_str = time_str + "," + str(timing[i])
        val = val + "," + str(val_acc[i])
        tst = tst + "," + str(tst_acc[i])
    
    print(timing, file=logfile)
    print(val, file=logfile)
    print(tst, file=logfile)
    
    omp_timing = np.array(timing)
    omp_cum_timing = list(generate_cumulative_timing(omp_timing))
    omp_tst_acc = list(filter(tst_acc))   
    print("Total time taken by " + strategy + " = " + str(omp_cum_timing[-1]))
    logfile.close()

### Training GradMatch Strategy with different settings 

In [ ]:
datadir = '../../data'
data_name = 'mnist'
fraction = float(0.1)
num_epochs = int(300)
select_every = int(20)
feature = 'dss'
num_runs = 5  # number of random runs
learning_rate = 0.01
model_name = 'MnistNet'
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using Device:", device)

Using Device: cuda


In [ ]:
for run in range(num_runs):
  train_model(num_epochs, data_name, datadir, feature, model_name, fraction, select_every, learning_rate, run, device, 'GradMatch')

  0%|          | 0/300 [00:00<?, ?it/s]

Budget, fraction and N: 5400 0.1 54000
/content/drive/MyDrive/Results/mnist/GradMatch/dss/0.1/20
mnist_fraction:0.1_epochs:300_selEvery:20_variant_runs0
selEpoch: 0, Selection Ended at: 2021-01-24 12:47:18.391980


  0%|          | 1/300 [00:04<21:18,  4.27s/it]

Epoch: 1 Validation Accuracy:  0.9438333333333333 Test Accuracy:  0.9514 Time:  1.7078437805175781
selEpoch: 1, Selection Ended at: 2021-01-24 12:47:22.667104


  1%|          | 2/300 [00:08<21:27,  4.32s/it]

...

Epoch: 298 Validation Accuracy:  0.9908333333333333 Test Accuracy:  0.9933 Time:  1.5649940967559814
selEpoch: 298, Selection Ended at: 2021-01-24 14:59:50.754231


100%|█████████▉| 299/300 [24:59<00:03,  3.85s/it]

Epoch: 299 Validation Accuracy:  0.9908333333333333 Test Accuracy:  0.9933 Time:  1.5704774856567383
Per Element Training Gradient Computation is Completed
Per Element Training Gradient Computation is Completed
Per Element Training Gradient Computation is Completed
Per Element Training Gradient Computation is Completed
Per Element Training Gradient Computation is Completed
Per Element Training Gradient Computation is Completed
Per Element Training Gradient Computation is Completed
Per Element Training Gradient Computation is Completed
Per Element Training Gradient Computation is Completed
Per Element Training Gradient Computation is Completed
OMP algorithm Subset Selection time is:  21.108144283294678
selEpoch: 299, Selection Ended at: 2021-01-24 15:00:15.690584


100%|██████████| 300/300 [25:24<00:00,  5.08s/it]

Epoch: 300 Validation Accuracy:  0.9908333333333333 Test Accuracy:  0.9933 Time:  22.72860813140869
GradMatch Selection Run---------------------------------
Final SubsetTrn: 3.782821573559886
Validation Loss and Accuracy: 0.17862302251160145 0.9918333333333333
Test Data Loss and Accuracy: 0.24376548023428768 0.9933
-----------------------------------
Total time taken by GradMatch = 0.2324228431781133


In [ ]:
for run in range(num_runs):
  train_model(num_epochs, data_name, datadir, feature, model_name, fraction, select_every, learning_rate, run, device, 'GradMatch-Explore')

Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw


Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw




Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw


Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw
Processing...
Done!
Budget, fraction and N: 5400 0.1 54000


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)
  0%|          | 0/300 [00:00<?, ?it/s]

...

Epoch: 298 Validation Accuracy:  0.9905 Test Accuracy:  0.9932 Time:  1.7189650535583496
selEpoch: 298, Selection Ended at: 2021-01-24 11:48:35.470440


100%|█████████▉| 299/300 [25:21<00:04,  4.32s/it]

Epoch: 299 Validation Accuracy:  0.9905 Test Accuracy:  0.9932 Time:  1.702911138534546
Per Element Training Gradient Computation is Completed
Per Element Training Gradient Computation is Completed
Per Element Training Gradient Computation is Completed
Per Element Training Gradient Computation is Completed
Per Element Training Gradient Computation is Completed
Per Element Training Gradient Computation is Completed
Per Element Training Gradient Computation is Completed
Per Element Training Gradient Computation is Completed
Per Element Training Gradient Computation is Completed
Per Element Training Gradient Computation is Completed
OMP algorithm Subset Selection time is:  23.39916205406189
selEpoch: 299, Selection Ended at: 2021-01-24 11:49:03.186087


100%|██████████| 300/300 [25:49<00:00,  5.17s/it]

Epoch: 300 Validation Accuracy:  0.9905 Test Accuracy:  0.9932 Time:  25.089895725250244
GradMatch-Explore Selection Run---------------------------------
Final SubsetTrn: 4.3814153833550336
Validation Loss and Accuracy: 0.221070297062397 0.9913333333333333
Test Data Loss and Accuracy: 0.24589406268205494 0.9937
-----------------------------------
Total time taken by GradMatch-Explore = 0.21725774447123208


In [ ]:
for run in range(num_runs):
  train_model(num_epochs, data_name, datadir, feature, model_name, fraction, select_every, learning_rate, run, device, 'GradMatchPB')

  0%|          | 0/300 [00:00<?, ?it/s]

Budget, fraction and N: 5400 0.1 54000
/content/drive/MyDrive/Results/mnist/GradMatchPB/dss/0.1/20
mnist_fraction:0.1_epochs:300_selEvery:20_variant_runs0
selEpoch: 0, Selection Ended at: 2021-01-24 11:49:07.535910


  0%|          | 1/300 [00:04<21:13,  4.26s/it]

Epoch: 1 Validation Accuracy:  0.9406666666666667 Test Accuracy:  0.9422 Time:  1.6854002475738525
selEpoch: 1, Selection Ended at: 2021-01-24 11:49:11.796621


  1%|          | 2/300 [00:08<21:07,  4.25s/it]

...

Epoch: 298 Validation Accuracy:  0.9928333333333333 Test Accuracy:  0.9925 Time:  1.7042310237884521
selEpoch: 298, Selection Ended at: 2021-01-24 13:55:33.103722


100%|█████████▉| 299/300 [24:47<00:04,  4.25s/it]

Epoch: 299 Validation Accuracy:  0.9928333333333333 Test Accuracy:  0.9925 Time:  1.6805696487426758
Per Element Training Gradient Computation is Completed
OMP algorithm Subset Selection time is:  15.448818445205688
selEpoch: 299, Selection Ended at: 2021-01-24 13:55:52.797552


100%|██████████| 300/300 [25:06<00:00,  5.02s/it]

Epoch: 300 Validation Accuracy:  0.9928333333333333 Test Accuracy:  0.9925 Time:  17.181947231292725
GradMatchPB Selection Run---------------------------------
Final SubsetTrn: 4.835538740202537
Validation Loss and Accuracy: 0.16209659539163113 0.9936666666666667
Test Data Loss and Accuracy: 0.24891766562359408 0.9928
-----------------------------------
Total time taken by GradMatchPB = 0.20470593088203007


In [ ]:
for run in range(num_runs):
  train_model(num_epochs, data_name, datadir, feature, model_name, fraction, select_every, learning_rate, run, device, 'GradMatchPB-Explore')

  0%|          | 0/300 [00:00<?, ?it/s]

Budget, fraction and N: 5400 0.1 54000
/content/drive/MyDrive/Results/mnist/GradMatchPB-Explore/dss/0.1/20
mnist_fraction:0.1_epochs:300_selEvery:20_variant_runs0
selEpoch: 0, Selection Ended at: 2021-01-24 13:55:57.242898


  0%|          | 1/300 [00:04<21:14,  4.26s/it]

Epoch: 1 Validation Accuracy:  0.9371666666666667 Test Accuracy:  0.9475 Time:  1.6865463256835938
selEpoch: 1, Selection Ended at: 2021-01-24 13:56:01.506499


  1%|          | 2/300 [00:08<21:11,  4.27s/it]

...

Epoch: 298 Validation Accuracy:  0.9901666666666666 Test Accuracy:  0.9923 Time:  1.675412654876709
selEpoch: 298, Selection Ended at: 2021-01-24 15:55:18.127827


100%|█████████▉| 299/300 [23:33<00:04,  4.24s/it]

Epoch: 299 Validation Accuracy:  0.9901666666666666 Test Accuracy:  0.9923 Time:  1.710378646850586
Per Element Training Gradient Computation is Completed
OMP algorithm Subset Selection time is:  15.325281858444214
selEpoch: 299, Selection Ended at: 2021-01-24 15:55:37.765863


100%|██████████| 300/300 [23:53<00:00,  4.78s/it]

Epoch: 300 Validation Accuracy:  0.9901666666666666 Test Accuracy:  0.9923 Time:  17.00560188293457
GradMatchPB-Explore Selection Run---------------------------------
Final SubsetTrn: 5.6856415175225266
Validation Loss and Accuracy: 0.26171065866947174 0.9906666666666667
Test Data Loss and Accuracy: 0.2395620010793209 0.9928
-----------------------------------
Total time taken by GradMatchPB-Explore = 0.1861075230439504


### Training using Full MNIST dataset

In [ ]:
num_runs = 2  # number of random runs
for run in range(num_runs):
  train_model(num_epochs, data_name, datadir, feature, model_name, fraction, select_every, learning_rate, run, device, 'Full')

Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw


Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw




Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw


Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw
Processing...
Done!
/content/drive/MyDrive/Results/mnist/Full/dss


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


...

Epoch: 298 Validation Accuracy:  0.9931666666666666 Test Accuracy:  0.9932 Time:  15.005432844161987


100%|█████████▉| 299/300 [1:27:08<00:17, 17.38s/it]

Epoch: 299 Validation Accuracy:  0.9931666666666666 Test Accuracy:  0.9933 Time:  15.195353031158447


100%|██████████| 300/300 [1:27:25<00:00, 17.49s/it]

Epoch: 300 Validation Accuracy:  0.9931666666666666 Test Accuracy:  0.9933 Time:  14.998042106628418
Full Selection Run---------------------------------
Final SubsetTrn: 15.260627160738295
Validation Loss and Accuracy: 0.15892674587666988 0.9938333333333333
Test Data Loss and Accuracy: 0.2220085341250524 0.994
-----------------------------------
Total time taken by Full = 1.2589929070075352


In [ ]:
num_runs = 3  # number of random runs
for run in range(num_runs):
  train_model(num_epochs, data_name, datadir, feature, model_name, fraction, select_every, learning_rate, run, device, 'Full')

  0%|          | 0/300 [00:00<?, ?it/s]

/content/drive/MyDrive/Results/mnist/Full/dss
mnist_fraction:None_epochs:300_selEvery:None_variant_runs0


  0%|          | 1/300 [00:17<1:26:08, 17.29s/it]

Epoch: 1 Validation Accuracy:  0.9808333333333333 Test Accuracy:  0.9812 Time:  14.932515621185303


  1%|          | 2/300 [00:34<1:25:51, 17.29s/it]

...

Epoch: 298 Validation Accuracy:  0.9926666666666667 Test Accuracy:  0.9926 Time:  15.208447217941284


100%|█████████▉| 299/300 [1:27:23<00:17, 17.45s/it]

Epoch: 299 Validation Accuracy:  0.9926666666666667 Test Accuracy:  0.9926 Time:  14.970836639404297


100%|██████████| 300/300 [1:27:41<00:00, 17.54s/it]

Epoch: 300 Validation Accuracy:  0.9926666666666667 Test Accuracy:  0.9926 Time:  14.974010229110718
Full Selection Run---------------------------------
Final SubsetTrn: 15.317540958275458
Validation Loss and Accuracy: 0.17077491246163845 0.9938333333333333
Test Data Loss and Accuracy: 0.22542105527827516 0.9936
-----------------------------------
Total time taken by Full = 1.263102642496427


<code style="color:#1F45FC;">
    <h3> <strong> Training has never been faster! </strong>  </h3>
</code>

Results show that there is ~6x speedup with 10% subset of the data with a loss of only 0.08% in accuracy. This timing also includes the subset selection time, thereby making the comparison fair.